In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch
from tqdm import tqdm

/Users/motiwari/miniforge3/envs/gpt2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load dataset and model from huggingface
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name).eval()
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [5]:
# Prepare input tokens
stride = 32
max_length = 1024
text = dataset["test"]["text"]
inputs = tokenizer(text, return_tensors="pt", max_length=max_length, stride=stride, truncation=True)

# Remove potential empty sequences
non_empty_indices = [i for i, input_ids in enumerate(inputs["input_ids"]) if len(input_ids) > 0]
inputs["input_ids"] = inputs["input_ids"][non_empty_indices]
inputs["attention_mask"] = inputs["attention_mask"][non_empty_indices]

# Compute the perplexity over the tokenized test set
total_loss = 0.0
num_chunks = len(inputs["input_ids"])

with torch.no_grad():
    for i in tqdm(range(num_chunks)):
        input_ids = inputs["input_ids"][i].unsqueeze(0)
        attention_mask = inputs["attention_mask"][i].unsqueeze(0)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        total_loss += outputs.loss.item()

avg_loss = total_loss / num_chunks
perplexity = torch.exp(torch.tensor(avg_loss)).item()

print(f"Perplexity: {perplexity}")


0it [00:00, ?it/s]


ZeroDivisionError: float division by zero

In [17]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast


model_id = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [18]:
from datasets import load_dataset

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


In [22]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    print(end_loc, begin_loc)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

  0%|                                                                                                                                                                                       | 0/562 [00:00<?, ?it/s]

1024 0


  0%|▎                                                                                                                                                                              | 1/562 [00:00<06:05,  1.53it/s]

1536 512


  0%|▌                                                                                                                                                                              | 2/562 [00:01<05:55,  1.58it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 559/562 [05:46<00:01,  1.62it/s]

287232 286208


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 560/562 [05:47<00:01,  1.63it/s]

287644 286720


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 560/562 [05:48<00:01,  1.61it/s]


In [20]:
ppl = torch.exp(torch.stack(nlls).mean())

In [21]:
ppl

tensor(25.1876)